<a href="https://colab.research.google.com/github/denklewer/ray-custom-agents/blob/master/trpo_ray.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!sudo apt-get install -y build-essential curl unzip psmisc
!pip install cython==0.29.0
!git clone https://github.com/ray-project/ray.git
!ray/ci/travis/install-bazel.sh
!pip install lz4
!mv ray ray-distr
!pip install -e ray-distr/python/. --verbose  # Add --user if you see a permission denied error.

Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.4ubuntu1).
unzip is already the newest version (6.0-21ubuntu1).
curl is already the newest version (7.58.0-2ubuntu3.6).
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  psmisc
0 upgraded, 1 newly installed, 0 to remove and 7 not upgraded.
Need to get 52.5 kB of archives.
After this operation, 266 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 psmisc amd64 23.1-1ubuntu0.1 [52.5 kB]
Fetched 52.5 kB in 1s (88.7 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back t

In [0]:
import sys
import os
sys.path.append(os.path.join("", '/content/ray-distr/python')) # To find local version of the library

In [3]:

!rm -r ray-custom-agents
!git clone https://github.com/denklewer/ray-custom-agents.git


rm: cannot remove 'ray-custom-agents': No such file or directory
Cloning into 'ray-custom-agents'...
remote: Enumerating objects: 88, done.
remote: Counting objects: 100% (88/88), done.
remote: Compressing objects: 100% (61/61), done.
remote: Total 88 (delta 38), reused 70 (delta 25), pack-reused 0
Unpacking objects: 100% (88/88), done.


In [0]:

sys.path.append(os.path.join("", '/content/ray-custom-agents')) # To find local version of the library


In [0]:
from trpo.trainer import DEFAULT_CONFIG,TRPOTrainer

In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import gym
import ray
from ray.tune.logger import pretty_print



In [7]:
ray.init(ignore_reinit_error=True, load_code_from_local=True)

2019-05-18 19:21:14,034	WARNING worker.py:1341 -- WARNING: Not updating worker name since `setproctitle` is not installed. Install this with `pip install setproctitle` (or ray[debug]) to enable monitoring of worker processes.
2019-05-18 19:21:14,037	INFO node.py:498 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-05-18_19-21-14_036626_125/logs.
2019-05-18 19:21:14,157	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:48284 to respond...
2019-05-18 19:21:14,292	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:35180 to respond...
2019-05-18 19:21:14,295	INFO services.py:806 -- Starting Redis shard with 2.58 GB max memory.
2019-05-18 19:21:14,340	INFO node.py:512 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-05-18_19-21-14_036626_125/logs.
2019-05-18 19:21:14,343	INFO services.py:1442 -- Starting the Plasma object store with 3.87 GB memory using /dev/shm.


{'node_ip_address': '172.28.0.2',
 'object_store_address': '/tmp/ray/session_2019-05-18_19-21-14_036626_125/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2019-05-18_19-21-14_036626_125/sockets/raylet',
 'redis_address': '172.28.0.2:48284',
 'session_dir': '/tmp/ray/session_2019-05-18_19-21-14_036626_125',
 'webui_url': None}

In [8]:
config = DEFAULT_CONFIG.copy()
config['num_workers'] = 2
config['num_sgd_iter'] = 30
config['num_gpus'] = 3
config['sgd_minibatch_size'] = 128
config['model']['fcnet_hiddens'] = [100, 100]
config['num_cpus_per_worker'] = 0  # This avoids running out of resources in the notebook environment when this cell is re-executed

agent = TRPOTrainer(config, 'CartPole-v0')

FYI: By default, the value function will not share layers with the policy model ('vf_share_layers': False).
2019-05-18 19:21:34,429	INFO policy_evaluator.py:312 -- Creating policy evaluation worker 0 on CPU (please ignore any CUDA init errors)


Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.random.categorical instead.
Instructions for updating:
Use tf.cast instead.


/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


2019-05-18 19:21:35,945	INFO policy_evaluator.py:732 -- Built policy map: {'default_policy': <trpo.trpo_policy_graph.TRPOPolicyGraph object at 0x7f6e7f6f74e0>}
2019-05-18 19:21:35,947	INFO policy_evaluator.py:733 -- Built preprocessor map: {'default_policy': <ray.rllib.models.preprocessors.NoPreprocessor object at 0x7f6e7f6f7198>}
2019-05-18 19:21:35,949	INFO policy_evaluator.py:344 -- Built filter map: {'default_policy': <ray.rllib.utils.filter.NoFilter object at 0x7f6e7f61a048>}
2019-05-18 19:21:36,006	INFO multi_gpu_optimizer.py:80 -- LocalMultiGPUOptimizer devices ['/gpu:0', '/gpu:1', '/gpu:2']


(pid=11726) WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/compat/compat.py:175: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=11726) Instructions for updating:
(pid=11726) non-resource variables are not supported in the long term
(pid=11727) WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/compat/compat.py:175: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=11727) Instructions for updating:
(pid=11727) non-resource variables are not supported in the long term
(pid=11726) 2019-05-18 19:21:41,039	INFO policy_evaluator.py:312 -- Creating policy evaluation worker 1 on CPU (please ignore any CUDA init errors)
(pid=11726) 2019-05-18 19:21:41.065532: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2200000000 Hz
(pid=11726) 2019-05-18 19

In [10]:
for i in range(2):
    result = agent.train()
    print(pretty_print(result))

(pid=11726) 2019-05-18 19:22:07,265	INFO policy_evaluator.py:438 -- Generating sample batch of size 200
(pid=11726) 2019-05-18 19:22:07,266	INFO sampler.py:308 -- Raw obs from env: { 0: { 'agent0': np.ndarray((4,), dtype=float64, min=-0.048, max=0.037, mean=-0.014)}}
(pid=11726) 2019-05-18 19:22:07,266	INFO sampler.py:309 -- Info return from env: {0: {'agent0': None}}
(pid=11726) 2019-05-18 19:22:07,266	INFO sampler.py:407 -- Preprocessed obs: np.ndarray((4,), dtype=float64, min=-0.048, max=0.037, mean=-0.014)
(pid=11726) 2019-05-18 19:22:07,266	INFO sampler.py:411 -- Filtered obs: np.ndarray((4,), dtype=float64, min=-0.048, max=0.037, mean=-0.014)
(pid=11726) 2019-05-18 19:22:07,267	INFO sampler.py:525 -- Inputs to compute_actions():
(pid=11726) 
(pid=11726) { 'default_policy': [ { 'data': { 'agent_id': 'agent0',
(pid=11726)                                   'env_id': 0,
(pid=11726)                                   'info': None,
(pid=11726)                                   'obs': np

2019-05-18 19:22:10,333	INFO multi_gpu_impl.py:146 -- Training on concatenated sample batches:

{ 'inputs': [ np.ndarray((4000, 4), dtype=float32, min=-2.671, max=2.54, mean=0.01),
              np.ndarray((4000,), dtype=float32, min=0.996, max=51.984, mean=12.926),
              np.ndarray((4000,), dtype=float32, min=-1.266, max=4.144, mean=-0.0),
              np.ndarray((4000,), dtype=int64, min=0.0, max=1.0, mean=0.5),
              np.ndarray((4000, 2), dtype=float32, min=-0.008, max=0.007, mean=-0.0),
              np.ndarray((4000,), dtype=float32, min=-0.005, max=0.005, mean=-0.0),
              np.ndarray((4000,), dtype=int64, min=0.0, max=1.0, mean=0.477),
              np.ndarray((4000,), dtype=float32, min=0.0, max=1.0, mean=0.956)],
  'placeholders': [ <tf.Tensor 'default_policy/obs:0' shape=(?, 4) dtype=float32>,
                    <tf.Tensor 'default_policy/value_targets:0' shape=(?,) dtype=float32>,
                    <tf.Tensor 'default_policy/advantages:0' shape=(?,

custom_metrics: {}
date: 2019-05-18_19-22-15
done: false
episode_len_mean: 22.977011494252874
episode_reward_max: 78.0
episode_reward_mean: 22.977011494252874
episode_reward_min: 9.0
episodes_this_iter: 174
episodes_total: 174
experiment_id: 9ae62e6c9087415bb982b06d080c2260
hostname: 202c4c6687c7
info:
  grad_time_ms: 4804.961
  learner:
    default_policy:
      cur_kl_coeff: 0.19999995827674866
      cur_lr: 4.999999873689376e-05
      entropy: 0.6644614934921265
      kl: 0.029476119205355644
      policy_loss: -0.04188487306237221
      total_loss: 176.8184814453125
      vf_explained_var: 0.02176438644528389
      vf_loss: 176.8544921875
  load_time_ms: 138.061
  num_steps_sampled: 4000
  num_steps_trained: 3906
  sample_time_ms: 3111.643
  update_time_ms: 1341.804
iterations_since_restore: 1
node_ip: 172.28.0.2
num_healthy_workers: 2
num_metric_batches_dropped: 0
off_policy_estimator: {}
pid: 125
policy_reward_mean: {}
sampler_perf:
  mean_env_wait_ms: 0.0887114286492576
  mean_i